Given is a set of 19 stores from pharmacy-consmetics-hygiene sector in Portugal. They are evaluated using 5 inputs and 2 outputs descibled below. We need to analyse their efficiency.

Inputs
- i1: Average stock - Average value of inventory (in euros); 
- i2: Number of employees - Number of full-time equivalent employees; 
- i3: Salary costs - Total cost with salaries (in euros);
- i4: Rent - Cost of space (in euros);
- i5: Area - Total area of the store (m^2)

Outputs:
- o1: Global sales - Global value of sales (in euros);
- o2: Family 4 sales/Global sales - Proportion of Family 4 products in global sales

Load necessry package

In [1]:
library(lpSolveAPI)

### Inputs

In [2]:
inputs <- read.csv("inputs.csv", sep=";")
# inputs = inputs[1:7, ]
inputs

X,i1,i2,i3,i4,i5
<fct>,<int>,<dbl>,<int>,<int>,<int>
s1,360614,13.2,153071,275240,213
s2,263736,9.5,111409,117916,213
s3,628938,17.8,218122,492305,436
s4,479582,16.5,189495,134824,262
s5,600449,15.9,222567,411982,331
s6,299876,12.3,159338,185368,208
s7,171010,9.2,92436,124355,231
s8,354506,13.9,153228,231525,400
s9,521819,13.1,155918,145527,222


### Outputs

In [3]:
outputs <- read.csv("outputs.csv", sep=";")
# outputs = outputs[1:7, ]
outputs

X,o1,o2
<fct>,<int>,<dbl>
s1,1994652,36.4
s2,1194289,44.6
s3,3841226,32.2
s4,2299879,23.8
s5,3905880,39.7
s6,1554821,37.2
s7,625315,22.0
s8,1570432,24.4
s9,2249522,28.0


# BCC
## Input-oriented DUAL model

In [4]:
createModel = function()
{
    return(make.lp(ncol = 1 + nrow(inputs))) #variables 1 for theta, 2:20 - lambdas
}
model <- createModel()

Set model's optimization direction.

In [5]:
setDirection = function(model) 
{
    lp.control(model, sense="min")
}

Model objective function.

In [6]:
setObjective = function(model, inputs, outputs)
{
    set.objfn(model, c(1), indices = c(1)) 
}

Combination of inputs not greater than analyzed DMU multiplied by $\theta$

In [7]:
addConstraintsWithTheta = function(model, inputs, outputs, storeID)
{
    for(i in 2:ncol(inputs))
    {
        constraint_coeffs = array(0, dim=(1+nrow(inputs)))
        constraint_coeffs[1] = -1*inputs[storeID, i]
        constraint_coeffs[2:(1+nrow(inputs))] = unlist(c(inputs[, i]))
        add.constraint(model, constraint_coeffs, "<=", 0) 
    }
}

Combination of outputs not less than analyzed DMU

In [8]:
addOutputNotLessConstraints = function(model, inputs, outputs, storeID)
{
    for(i in 2:ncol(outputs)) # one constraint for each DMU
    {
        constraint_coeffs = array(0, dim=(1+nrow(inputs)))
        constraint_coeffs[2:(1+nrow(inputs))] = unlist(c(outputs[, i]))
        add.constraint(model, constraint_coeffs, ">=", outputs[storeID, i])
    }
}

Combination should be convex -- lambdas must sum to 1

In [9]:
addLambdasMustSumToOneConstraint = function(model, inputs, outputs)
{
    constraint_coeffs = array(1, dim=(1+nrow(inputs)))
    constraint_coeffs[1] = 0
    add.constraint(model, constraint_coeffs, "=", 1)
}

Function prepareModel creates model and adds all constraints

In [10]:
prepareModel = function(inputs, outputs, storeId)
{
    model = createModel()
    setDirection(model)
    setObjective(model, inputs, outputs)
    addConstraintsWithTheta(model, inputs, outputs, storeId)
    addOutputNotLessConstraints(model, inputs, outputs, storeId)
    addLambdasMustSumToOneConstraint(model, inputs, outputs)
    
    return(model)
}

Save model to file (large model is not printed)

In [11]:
model = prepareModel(inputs, outputs, 1)
print(model)
write.lp(model, filename="model.lp")

Model name: 
  a linear program with 20 decision variables and 8 constraints


In [12]:
inputs <- read.csv("inputs.csv", sep=";")
outputs <- read.csv("outputs.csv", sep=";")

for(storeId in 1:nrow(inputs))
{
    model = prepareModel(inputs, outputs, storeId)
    solve(model)
    
    efficiency = get.objective(model)
    cat("Store", storeId)
    cat("\tInputs:  ")
    for (i in inputs[storeId, 2:ncol(inputs)]){
        cat(i, "\t")
    }
    cat("\n\tOutputs: ")
    for (i in outputs[storeId, 2:ncol(outputs)]){
        cat(i, "\t")
    }
    
    
    cat("\n\n>>>Efficiency:", efficiency, "\n\n", sep="")
    vars = get.variables(model) #first - theta, rest - lambdas 
    for (i in 2:length(vars)) {
        if(vars[i] > 0){
            cat("lambda_", i-1, ": ",vars[i], "\n", sep="")
        }
    }
    
    
    if(efficiency < 0.9999999) #not 1, because of numerical precision error 
    {
        cat("\nStore is not efficient, making efficient convex combination\n\n")
        
        effective_inputs = array(0, length(inputs)-1)
        effective_outputs = array(0, length(outputs)-1) 
        
        for(inputIdx in 1:(length(inputs)-1)) {
            for(i in 2:length(vars)) {
                effective_inputs[inputIdx] = effective_inputs[inputIdx] + vars[i] * inputs[i-1, inputIdx+1]
            }
                
        }
            
        for(outputIdx in 1:(length(outputs)-1)) {
            for(i in 2:length(vars)) {
                effective_outputs[outputIdx] = effective_outputs[outputIdx] + vars[i] * outputs[i-1, outputIdx+1]
            }
        }
        cat("Effective inputs:", effective_inputs, "\t(should be better than original)\n")
        cat("Effective outputs:", effective_outputs, "\t\t\t\t(should not be worse than original)\n")
        
        reduce_inputs_by = inputs[storeId, 2:length(inputs)] - effective_inputs
        cat("\nIt should reduce inputs by:", paste(round(reduce_inputs_by, 2)), "\n")
    }
#     else {
#         cat("Store is efficient\n")
#     }
    
    cat("\n--------------------------------------------------------\n")
}

Store 1	Inputs:  360614 	13.2 	153071 	275240 	213 	
	Outputs: 1994652 	36.4 	

>>>Efficiency:0.9870769

lambda_10: 0.2322581
lambda_12: 0.1060539
lambda_13: 0.1298732
lambda_15: 0.4218086
lambda_19: 0.1100063

Store is not efficient, making efficient convex combination

Effective inputs: 355953.8 11.86999 151092.9 191140.2 210.2474 	(should be better than original)
Effective outputs: 1994652 36.4 				(should not be worse than original)

It should reduce inputs by: 4660.24 1.33 1978.15 84099.84 2.75 

--------------------------------------------------------
Store 2	Inputs:  263736 	9.5 	111409 	117916 	213 	
	Outputs: 1194289 	44.6 	

>>>Efficiency:1

lambda_2: 1

--------------------------------------------------------
Store 3	Inputs:  628938 	17.8 	218122 	492305 	436 	
	Outputs: 3841226 	32.2 	

>>>Efficiency:0.8110404

lambda_12: 0.6515974
lambda_18: 0.3484026

Store is not efficient, making efficient convex combination

Effective inputs: 510094.2 14.26703 170361 246851.3 293.8324 